# Tutorial

<b>Warning: it is necessary to use <u>setups='sg15'</u> in GPAW</b> (ONCV pseudopotentials)

In [1]:
from gpaw import GPAW, FermiDirac
from gpaw.wavefunctions.pw import PW
from ase.io import read
from TDSE import TDSE
import numpy as np
from tqdm import tqdm
from ase.units import Hartree, Bohr
import matplotlib.pyplot as plt
import os
%matplotlib inline

c = 20
PW_cut=600
atoms = read('hBN.cif')
atoms.cell[2,2]=c
atoms.center()

calc = GPAW(mode=PW(PW_cut),xc='PBE',
            kpts={'size': (24, 24, 1)},symmetry='off',
            setups='sg15',
            occupations=FermiDirac(0.0001))

atoms.set_calculator(calc)
atoms.get_potential_energy()
calc.write('gs_sg15.gpw')


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  1.5.1
 |___|_|             

User:   drg@forsite
Date:   Thu Apr 11 15:02:42 2019
Arch:   x86_64
Pid:    24115
Python: 3.6.7
gpaw:   /usr/local/lib/python3.6/dist-packages/gpaw
_gpaw:  /usr/local/lib/python3.6/dist-packages/
        _gpaw.cpython-36m-x86_64-linux-gnu.so
ase:    /home/drg/.local/lib/python3.6/site-packages/ase (version 3.17.0)
numpy:  /home/drg/.local/lib/python3.6/site-packages/numpy (version 1.16.2)
scipy:  /home/drg/.local/lib/python3.6/site-packages/scipy (version 1.2.1)
units:  Angstrom and eV
cores:  1

Input parameters:
  kpts: {size: (24, 24, 1)}
  mode: {ecut: 600.0,
         gammacentered: False,
         name: pw}
  occupations: {name: fermi-dirac,
                width: 0.0001}
  setups: sg15
  symmetry: off
  xc: PBE

System changes: positions, numbers, cell, pbc, initial_charges, initial_magmoms 

Initialize ...

Norm-conserving UPF setup:
  Element:    B
  Z:         

# Initialization 

In [2]:
tdse=TDSE(calc)

In [ ]:
dt=1;steps=10000;p=-8.0
time=np.arange(steps)*dt
A=lambda t:10**p*np.exp(-t/dt)*np.array([1,1,1])
tdse.propagate(dt=dt,steps=steps,A=A)

 79%|███████▊  | 7872/10000 [12:40<03:17, 10.76it/s]  

In [ ]:
for i in range(3):
    plt.figure()
    plt.plot(tdse.J[:100,i].real)
# plt.plot(tdse.J[:,2].imag)

In [ ]:
for i in range(3):
    spectrum=np.fft.fft(tdse.J[:,i])
    freq = np.fft.fftfreq(steps, d=dt)*2*np.pi*Hartree
    spectrum=spectrum[np.argsort(freq)]
    freq=np.sort(freq)
    spectrum=np.abs(spectrum*freq)**2
    from scipy.ndimage.filters import gaussian_filter1d

    plt.figure(figsize=(12,8))
    plt.plot(freq,gaussian_filter1d(spectrum,1),'-')
    plt.xlim([0,20])
    plt.ylim([0,None])